In [ ]:
# %%
""" 
Three cases used by the model: winter (case1), fall turnover (case 2), stratified season (case 3). 
For each case, computation of the temperatures, solids concentration and thermocline 
depth at the next time step.  
 
TO FILL! 
"""

import numpy as np
import sys
from lake_functions import surfheat,alphaT,rho_TCss, seasonal_therm
from statistics import mean

from lake_model import Css_epi

# %% ###########################################################################
def case1(VAR_LAKE,Twinter=4,hmax=8):
    """ 
    Winter case: ice-covered lake, no heat fluxes = constant temperatures  
    and concentrations.  
      
    INPUTS: 
        VAR_LAKE: dictionary containing 
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C] 
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L] 
            h_epi: thermocline depth [m] 
        Twinter: temperature of maximum density [°C] 
        hmax: lake depth [m] 
         
    OUTPUTS: 
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step 
    """

    # -------- TO FILL --------
    # Temperature
    T_epi1 = Twinter
    T_hypo1 = Twinter
    # solid concentrations
    Css_epi1 = VAR_LAKE["Css_epi"]
    Css_hypo1 = VAR_LAKE["Css_hypo1"]

    h_epi1 = hmax
    
    VAR_LAKE={"T_epi":T_epi1,"T_hypo":T_hypo1,"Css_epi":Css_epi1,"Css_hypo":Css_hypo1,"h_epi":h_epi1}

    return VAR_LAKE
# %% ###########################################################################
def case2(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,C_FFT,Ta, Wsp, RH, P, C, hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6):
    """ 
    Fall turnover case: one box with heat fluxes. 
      
    INPUTS: 
        VAR_LAKE: dictionary containing 
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C] 
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L] 
            h_epi: thermocline depth [m] 
        Hsurf: surface heat flux [W/m^2] 
        Hsw0: surface radiation flux [W/m^2] 
        tyear: time as DOY 
        iceon: iceon date as DOY 
        Vs: settling velocity [m/s] 
        C_FFT: solids concentration in the FFT [g/m^3] 
        hmax: lake depth [m] 
        g: gravitational acceleration [m/s^2] 
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)] 
        Twinter: temperature of maximum density [°C] 
        A0: lake surface area [m^2] 
         
         
    OUTPUTS: 
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step 
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date) 
    """
   # -------- TO FILL --------
    
   # ----Initial setup----
    h_epi2 = hmax  # Thermocline depth
    T_epi2 = VAR_LAKE["T_epi"]  # Temperature, constant
    Css_epi2 = VAR_LAKE["Css_epi"] # Initial solid concentration constant
    Css_hypo2 = VAR_LAKE["Css_epi"]
    rho_w2 = rho_TCss(T_epi2, Css_epi2) # total density
    rho_fft = rho_TCss(T_epi2, 3*10**5, rho_w2) # calculation of average rho of FFT, BUT NOT SURE
    m_w2 = rho_w2*A0*hmax # water mass
    
    # ----Special process in fall----
    # Heat flux and temperature change CALCULATE OUTSIDE IN MAIN CODE
    Hsurf = surfheat(T_epi2,Ta,Wsp,RH,P,C)
    Qnet2 = -(Hsurf + Hsw0)*A0
    dT2 = (Qnet2/(m_w2*Cpw))/24*60*60
    Tnew2 = T_epi2 + dT2

    # --turbulent mixing--
    C_sed = -Css_epi2*Vs*A0*24*60*60/A0*hmax
    # --Redistribution--
    alpha2 = alphaT(T_epi2)
    kd = 0.325 + 66 * Css_epi2 / 1000  # mg/L to g/L if needed
    B = alpha2*g*(Hsurf + Hsw0*(1-np.exp(-kd*hmax))) / (rho_w2 * Cpw)
    # velocity
    rho_30 = rho_TCss(T_epi2, 30)
    N_fft2 = g*(rho_30-rho_w2)/(0.5*rho_fft) 

    # --stability and concentration calculate--
    if B > 0: #unstable, fully mixed, epi depth equals hmax, hypolimnion depth also h max
        # newly calculated temp: Hsw consider different layer
        # Tepi_new = T_epi + Hepi * 24/3600/(Cp*rhoepi*hepinew)
        # Thypo-new = Tepi_new(fully mixed)
        if Tepi_new <= 4: #additional safety
            if t[k] > 200:
                iceon = t[k]
            else:
                print("Warning, ice in summer!")
            # Tepi_new = Twinter
            # ...
        else: # no ice, calculate velocities, calculate N2, then we will have new mass and concentration
            # still put some safety checks (Css should be positive, Vs as well)
            # Cssepi = Csshypo
    else: # stable, we will go from mixed to stratified
        # calculate thermal structure
        # linear temperature, so T_avg is T_surf - T_hypo/2
        # this time Hepi is really close to the bottom,

    
    C_red = C_FFT*Vr*A0*24*60*60/A0*hmax
    Cssnew2 = Css_epi2 + C_sed + C_red
    
    # iceon (not sure)
    if T_epi2 <= Twinter:
        iceon = tyear

    VAR_LAKE={"T_epi":Tnew2,"T_hypo":Tnew2,"Css_epi":Css_epi2,"Css_hypo":Css_hypo2,"h_epi":h_epi2}
    return VAR_LAKE, iceon
# %% ###########################################################################
def case3(VAR_LAKE,Hsurf,Hsw_epi,Hsw0,H_diff, B_flux, N2, tyear,iceon,Vs,ustar,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6,Dth=1.4E-7):
    """ 
    Summer Stratification case: two boxes with vertical processes. 
      
    INPUTS: 
        VAR_LAKE: dictionary containing 
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C] 
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L] 
            h_epi: thermocline depth [m] 
        Hsurf: surface heat flux [W/m^2] 
        Hsw0: surface radiation flux [W/m^2] 
        tyear: time as DOY 
        iceon: iceon date as DOY 
        Vs: settling velocity [m/s] 
        ustar: friction velocity [m/s] 
        hmax: lake depth [m] 
        g: gravitational acceleration [m/s^2] 
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)] 
        Twinter: temperature of maximum density [°C] 
        A0: lake surface area [m^2] 
        Dth: thermal molecular diffusivity [m^2/s] 
         
   
    OUTPUTS: 
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step 
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date) 
    """
    # -------- TO FILL --------

    # QUESTIONS TO FAZEL:
    # 1. WHAT SHOULD STAY IN CASES3 AND WHAT SHOULD GO TO MAIN BODY OF THE CODE?

    # ---- initial setup ---
    T_epi = VAR_LAKE["T_epi"]
    T_hypo = VAR_LAKE["T_hypo"]
    Css_epi = VAR_LAKE["Css_epi"]
    Css_hypo = VAR_LAKE["Css_hypo"]
    h_epi = VAR_LAKE["h_epi"]

    # --- constants ---
    rho = 1000 # water density [kg/m^3] # the water density is not constant: rho_epi=rho_TCss(T_epi,Css_epi); rho_hypo=rho_TCss(T_hypo,Css_hypo)
    Cp = Cpw
    dt = 86400 # time step [s]

    # --- import necessary equations ---
    from lake_functions import rho_TCss, seasonal_therm # This can be moved above where you load all packages

    # --- Additional information: shortwave-radiative flux: beer-lambert law---
    c0 = 0.325
    c1 = 66  # m^-1 * (g/L)^-1
    kd = c0 + c1 * (Css_epi / 1000)  # Css in g/L
    Hsw_epi = Hsw0 * (1 - np.exp(-kd * h_epi))

    # --- Additional information: vertical diffusive heat flux ---
    Dth = 1.4e-7 # this is an input to "case3" function. You must not redefine it. please remove this line. 
    dTdz = (T_epi - T_hypo) / (0.5 * hmax)
    H_diff = -rho * Cp * (Dth + Kz) * dTdz # replace the rho here with average of rho_epi and eho_hypo

    # --- net heat fluxes ---
    H_epi_net = -Hsurf - Hsw_epi + H_diff
    H_hypo_net = - Hsw0 + Hsw_epi - H_diff

    # --- thermocline depth ---
    h_epi_new = seasonal_therm(prev_h_epi, lake_params["dt"], fluxes["B"], N2) # be careful with calling other functions inside another function.
    # For example here "prev_h_epi" has not beeen defined anywhere inside "case3" function. You also have not calculated N2 and Bflux.
    # You must replace it with something like:
    # first calculate N2 (using rho_epi and rho_hypo) and Bflux (using only rho_epi) . Then calculate 
    # h_epi_new = seasonal_therm(h_epi,dt, Bflux, N2)
    dh = h_epi_new - h_epi  # change in the thermocline depth
    dhdt = abs(dh / dt) # absolute value because only looks at velocity, not direction

    # --- convective velocity (only if B > 0) ---
    v_therm = dh / dt # velocity of thermocline change
    if B_flux > 0:
        v_conv = 0.6 * (B_flux * h_epi)**(1/3) # convective velocity
    else:
        v_conv = 0

    # --- settling velocity in epilimnion ---
    vs_epi = max(0, Vs - v_conv - abs(v_therm))
    vs_hypo = Vs

    # --- mass flux from sedimentation ---
    F_sed_epi = dt * vs_epi * A0 * Css_epi # sedimentation from epi to hypo [mg]

    # --- mass added due to thermocline motion ---
    if dh > 0: # thermocline deepens → hypo water moves to epi
        F_red = Css_hypo * A0 * dh
    else: # thermocline shallows → epi water moves to hypo
        F_red = Css_epi * A0 * abs(dh) # why abs(dh) and not dh? 

    # --- mass balances ---
    Mss_epi = Css_epi * h_epi * A0 - F_sed_epi + F_red
    Mss_hypo = Css_hypo * (hmax - h_epi) * A0 + F_sed_epi - F_red # it should be like: Css_hypo * (hmax - h_epi) * A0 + F_sed_epi - F_red-F_sed_hypo
    # where F_sed_hypo = dt * vs_hypo * A0 * Css_hypo
    # think carefully about the mass balance

    # --- Updated concentrations ---
    Css_epi_new = max(0, Mss_epi / (h_epi_new * A0))
    Css_hypo_new = max(0, Mss_hypo / ((hmax - h_epi_new) * A0))

    # --- updated temperatures ---
    T_epi_new = T_epi + H_epi_net * dt / (rho * Cp * h_epi_new) # use rho_epi
    T_hypo_new = T_hypo + H_hypo_net * dt / (rho * Cp * (hmax - h_epi_new)) # use rho_hypo


    # what if after calculation we get an unstable or mixed situation? what if we have some ice signal?
    # if rho_TCss(T_epi_new,Css_epi_new)>=rho_TCss(T_hypo_new,Css_hypo_new) or h_epi_new>(hmax-0.1): # Completely mixed: one box model
        # ...

    # if T_epi_new<=4:
        #if tyear>200:
            #....
        #else:
        #...

    # be careful how we define and call case3 function. You're not returning all the variables. I corrected it for you.
    VAR_LAKE_NEW={"T_epi":T_epi_new,"T_hypo":T_hypo_new,"Css_epi":Css_epi_new,"Css_hypo":Css_hypo_new,"h_epi":h_epi_new}
    
    return VAR_LAKE_NEW,iceon,F_red,F_sed_epi,F_sed_hypo

